# 01 — Single Person Demo

This notebook runs the Human3D pipeline on a **single-person** image and visualizes the main outputs:

- Depth map (`depth.npy`, `depth.png`)
- Pose overlay (`pose_overlay.png`)
- Segmentation mask and overlay (`seg_mask.png`, `seg_overlay.png`)
- Point cloud (`pointcloud.ply`)

It is meant to be readable and reproducible, not fancy.

In [ ]:
# --- Standard imports ---
import os
from pathlib import Path
import json
import numpy as np

# --- Visualization ---
import matplotlib.pyplot as plt

# Make plots bigger by default (no fixed colors here)
plt.rcParams["figure.figsize"] = (10, 6)

# --- Repo root detection (works when notebook is in ./notebooks) ---
REPO_ROOT = Path.cwd()
if (REPO_ROOT / "src").exists() and (REPO_ROOT / "configs").exists():
    pass
elif (REPO_ROOT.parent / "src").exists() and (REPO_ROOT.parent / "configs").exists():
    REPO_ROOT = REPO_ROOT.parent
else:
    raise RuntimeError("Could not locate repo root. Open this notebook from inside the repo folder.")

print("Repo root:", REPO_ROOT)

# Ensure src is importable
import sys
SRC_DIR = REPO_ROOT / "src"
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

# Imports from the project
from human3d.utils.config import load_config
from human3d.pipeline import Human3DPipeline

## 1) Choose an input image

Put an image in `data/` (or `data/sample_images/`) and set `INPUT_PATH` below.

**Tip:** For the best single-person result, use a full-body photo with the subject clearly visible and minimal occlusion.

In [ ]:
# Path to your single-person image (edit this)
INPUT_PATH = REPO_ROOT / "data" / "sample.jpg"

assert INPUT_PATH.exists(), f"Input image not found: {INPUT_PATH}"
print("Using input:", INPUT_PATH)

## 2) Load config and run the pipeline

In [ ]:
CFG_PATH = REPO_ROOT / "configs" / "pipeline.yaml"
cfg = load_config(str(CFG_PATH))

pipe = Human3DPipeline(cfg)
out_dir = Path(pipe.run(str(INPUT_PATH)))

print("Outputs saved to:", out_dir)

## 3) Inspect outputs

This section loads the output images and shows them inline.

In [ ]:
from PIL import Image

def show_img(p: Path, title: str):
    if not p.exists():
        print(f"[MISS] {p.name} not found")
        return
    img = Image.open(p)
    plt.figure()
    plt.imshow(img)
    plt.axis("off")
    plt.title(title)
    plt.show()

show_img(out_dir / "depth.png", "Depth visualization (MiDaS)")
show_img(out_dir / "pose_overlay.png", "Pose overlay (YOLOv8-Pose)")
show_img(out_dir / "seg_mask.png", "Segmentation mask (SAM)")
show_img(out_dir / "seg_overlay.png", "Segmentation overlay")

## 4) Load numeric depth and basic stats

In [ ]:
depth_path = out_dir / "depth.npy"
depth = np.load(depth_path)

print("depth shape:", depth.shape)
print("depth dtype:", depth.dtype)
print("depth min/max:", float(depth.min()), float(depth.max()))
print("depth nonzero ratio:", float((depth > 0).mean()))

## 5) Point cloud preview (optional)

Point clouds can be viewed with:

- **Open3D** (interactive window)
- **CloudCompare / MeshLab** (open `pointcloud.ply`)

If Open3D fails to open a window (common on some systems), just open the `.ply` file externally.

In [ ]:
pc_path = out_dir / "pointcloud.ply"
print("PLY:", pc_path)

try:
    import open3d as o3d
    pcd = o3d.io.read_point_cloud(str(pc_path))
    print(pcd)
    # Interactive viewer (may not work in some notebook environments)
    o3d.visualization.draw_geometries([pcd])
except Exception as e:
    print("Open3D preview failed:", repr(e))
    print("You can open the PLY file in CloudCompare or MeshLab instead.")